ENSF 444 Project - Group 85

In [140]:
import pandas as pd
import numpy as np

In [141]:
results_df = pd.read_csv('data/results.csv')
qualifying_df = pd.read_csv('data/qualifying.csv')
lap_times_df = pd.read_csv('data/lap_times.csv')
pit_stops_df = pd.read_csv('data/pit_stops.csv')
driver_standings_df = pd.read_csv('data/driver_standings.csv')
drivers_df = pd.read_csv('data/drivers.csv')
races_df = pd.read_csv('data/races.csv')
constructors_df = pd.read_csv('data/constructors.csv')
constructors_standings_df = pd.read_csv('data/constructor_standings.csv')
status_df = pd.read_csv('data/status.csv')

Transform Data

In [142]:
results_df.drop(['number','positionText','positionOrder','points','laps','time','milliseconds','fastestLap','rank','fastestLapTime','fastestLapSpeed'], axis=1, inplace=True)
results_df.rename(columns={'position':'final_position','grid':'starting_position'}, inplace=True)

results_df['final_position'] = results_df['final_position'].replace('\\N', np.nan)
results_df.final_position = pd.to_numeric(results_df.final_position)
print(results_df.dtypes)
results_df.head()

resultId               int64
raceId                 int64
driverId               int64
constructorId          int64
starting_position      int64
final_position       float64
statusId               int64
dtype: object


,resultId,raceId,driverId,constructorId,starting_position,final_position,statusId
0,1,18,1,1,1,1.0,1
1,2,18,2,2,5,2.0,1
2,3,18,3,3,7,3.0,1
3,4,18,4,4,11,4.0,1
4,5,18,5,1,3,5.0,1


In [143]:
qualifying_df.drop(['qualifyId','number','q1','q2','q3'], axis=1, inplace=True)
qualifying_df.rename(columns={'position':'qualifying_position'}, inplace=True)

print(qualifying_df.dtypes)
qualifying_df.head()

raceId                 int64
driverId               int64
constructorId          int64
qualifying_position    int64
dtype: object


,raceId,driverId,constructorId,qualifying_position
0,18,1,1,1
1,18,9,2,2
2,18,5,1,3
3,18,13,6,4
4,18,2,2,5


In [144]:
lap_times_df.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [145]:
pit_stops_df.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [146]:
driver_standings_df.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


In [147]:
drivers_df['driver_name'] = drivers_df['forename'] + ' ' + drivers_df['surname']
drivers_df.drop(['driverRef','number','code','forename','surname','dob','nationality','url'], axis=1, inplace=True)

print(drivers_df.dtypes)
drivers_df.head()

driverId        int64
driver_name    object
dtype: object


,driverId,driver_name
0,1,Lewis Hamilton
1,2,Nick Heidfeld
2,3,Nico Rosberg
3,4,Fernando Alonso
4,5,Heikki Kovalainen


In [148]:
races_df.drop(columns=['round','date','time','url','fp1_date','fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date','quali_time','sprint_date','sprint_time',], axis=1, inplace=True)

print(races_df.dtypes)
races_df.head()

raceId        int64
year          int64
circuitId     int64
name         object
dtype: object


,raceId,year,circuitId,name
0,1,2009,1,Australian Grand Prix
1,2,2009,2,Malaysian Grand Prix
2,3,2009,17,Chinese Grand Prix
3,4,2009,3,Bahrain Grand Prix
4,5,2009,4,Spanish Grand Prix


In [149]:
constructors_df.drop(columns=['constructorRef','nationality','url'], axis=1, inplace=True)

constructors_df.head()

,constructorId,name
0,1,McLaren
1,2,BMW Sauber
2,3,Williams
3,4,Renault
4,5,Toro Rosso


In [150]:
constructors_standings_df.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0


In [151]:
status_df.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine
